In [12]:
import pandas as pd
import numpy as np

In [77]:
loans = pd.DataFrame.from_csv('lending-club-data.csv')

d:\Users\zeyang\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2825: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [78]:
features = ['grade',              # grade of the loan
            'term',               # the term of the loan
            'home_ownership',     # home ownership status: own, mortgage or rent
            'emp_length',         # number of years of employment
           ]
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
del loans['bad_loans']
target = 'safe_loans'
loans = loans[features + [target]]
loans = loans.reset_index(drop=True)
loans = pd.get_dummies(loans)

In [79]:
train_data = loans.ix[pd.read_json('module-8-assignment-2-train-idx.json')[0]].reset_index(drop=True)

In [80]:
test_data = loans.ix[pd.read_json('module-8-assignment-2-test-idx.json')[0]].reset_index(drop=True)

In [127]:
def intermediate_node_weighted_mistakes(labels_in_node, data_weights):
    weighted_mistakes_all_positive = np.sum((labels_in_node != 1) * data_weights)
    
    weighted_mistakes_all_negative = np.sum((labels_in_node != -1) * data_weights)
    
    if weighted_mistakes_all_positive <= weighted_mistakes_all_negative:
        return weighted_mistakes_all_positive, 1
    return weighted_mistakes_all_negative, -1

In [128]:
# Quiz Question: If we set the weights α=1 for all data points, how is the weight of 
# mistakes WM(α,y^) related to the classification error?

# Identical

In [129]:
def best_splitting_feature(data, features, target, data_weights):
    best_feature = None
    best_error = float('+inf')
    
    for feature in features:
        left_split = data[data[feature] == 0]
        right_split = data[data[feature] == 1]
        
        #print('Left ' ,len(left_split))
        #print('Right ', len(right_split))
        #print('Total ', len(data_weights))
        left_data_weights = data_weights[np.where(data[feature] == 0)]
        right_data_weights = data_weights[np.where(data[feature] == 1)]
        #print(len(left_split[target].as_matrix()))
        #print(len(left_data_weights))
        left_weighted_mistakes, left_class = intermediate_node_weighted_mistakes(
            left_split[target].as_matrix(), left_data_weights)
        right_weighted_mistakes, right_class = intermediate_node_weighted_mistakes(
            right_split[target].as_matrix(), right_data_weights)
        weighted_error = (left_weighted_mistakes + right_weighted_mistakes) / np.sum(data_weights)
        
        if weighted_error < best_error:
            best_feature = feature
            best_error = weighted_error
            
        return best_feature

In [146]:
class Node:
    def __init__(self, is_leaf=False, prediction=None, left=None, right=None, splitting_feature=None):
        self.is_leaf = is_leaf
        self.prediction = prediction
        self.left = left
        self.right = right
        self.splitting_feature = splitting_feature

In [131]:
def create_leaf(target_values, data_weights):
    leaf = Node(is_leaf=True)
    weighted_error, best_class = intermediate_node_weighted_mistakes(target_values, data_weights)
    
    leaf.prediction = best_class
    return leaf

In [147]:
def weighted_decision_tree_create(data, features, target, data_weights, current_depth=1, max_depth=10):
    remaining_features = features[:]
    target_values = data[target]
    
    if (target_values == 1).all() or (target_values == -1).all():
        return create_leaf(target_values, data_weights)
    if len(remaining_features) == 0:
        return create_leaf(target_values, data_weights)
    if current_depth > max_depth:
        return create_leaf(target_values, data_weights)
    
    splitting_feature = best_splitting_feature(data, remaining_features, target, data_weights)
    remaining_features.remove(splitting_feature)
    
    left_split = data[data[splitting_feature] == 0]
    right_split = data[data[splitting_feature] == 1]
    
    left_data_weights = data_weights[np.where(data[splitting_feature] == 0)]
    right_data_weights = data_weights[np.where(data[splitting_feature] == 1)]
    
    if len(left_split) == len(data):
        return create_leaf(left_split[target], left_data_weights)
    if len(right_split) == len(data):
        return create_leaf(right_split[target], right_data_weights)
    
    left_tree = weighted_decision_tree_create(left_split, remaining_features, target, left_data_weights, 
                                              current_depth+1, max_depth)
    right_tree = weighted_decision_tree_create(right_split, remaining_features, target, right_data_weights,
                                              current_depth+1, max_depth)
    
    return Node(left=left_tree, right=right_tree, splitting_feature=splitting_feature)


In [148]:
def count_nodes(tree):
    if tree.is_leaf:
        return 1
    return 1 + count_nodes(tree.left) + count_nodes(tree.right)

In [169]:
def classify(tree, x, annotate = False):   
    # If the node is a leaf node.
    if tree.is_leaf:
        if annotate: 
            print("At leaf, predicting %s" % tree.prediction)
        return tree.prediction
    else:
        # Split on feature.
        split_feature_value = x[tree.splitting_feature]
        if annotate: 
            print("Split on %s = %s" % (tree.splitting_feature, split_feature_value))
        if split_feature_value == 0:
            return classify(tree.left, x, annotate)
        else:
            return classify(tree.right, x, annotate)

In [170]:
def evaluate_classification_error(tree, data, target):
    prediction = data.apply(lambda x: classify(tree, x), axis=1)
    return np.sum(prediction != data[target]) / float(len(data))

In [171]:
new_features = train_data.columns.values.tolist()
new_features.remove(target)

In [172]:
example_data_weights = np.zeros(len(train_data))
example_data_weights[:10] = 1
example_data_weights[-10:] = 1

In [173]:
subset_20 = weighted_decision_tree_create(train_data, new_features, target, example_data_weights, max_depth=2)

In [174]:
evaluate_classification_error(subset_20, train_data, target)

0.5414786159467011